# Optimizing Strategy Parameters

<table style="width:100%; height:90%">
      <tr>
    <th>Parametrize the Strategy</th>
    <th>Optimizing Limits' Parameters</th>
  </tr>
  <tr>
    <td><img src="src/07_Code_Regression Strategy Limits X.png" alt="Parametrize the Strategy" style="width:100%"></td>
    <td><img src="src/07_Table_Optimize BG Default Defaults.png" alt="Optimizing Limits' Parameters" style="width:100%"></td>
  </tr>
</table>

## Load the model

In [1]:
import pickle

with open('models/model_dt_regression.pkl', 'rb') as f:
    model_dt = pickle.load(f)

model_dt

DecisionTreeRegressor(max_depth=15)

## Load the data

In [2]:
import pandas as pd

df = pd.read_excel('data/Microsoft_LinkedIn_Processed.xlsx', index_col=0, parse_dates=['Date'])
df

,Close,High,Low,Open,Volume,change_tomorrow,change_tomorrow_direction
Date,,,,,,,
2016-12-08,55.181118,55.696664,55.027361,55.443412,21220800,1.549151,UP
2016-12-09,56.049408,56.067498,55.289661,55.334883,27349400,0.321693,UP
2016-12-12,56.230297,56.347878,55.823293,55.913737,20198100,1.286142,UP
2016-12-13,56.962921,57.360883,56.293622,56.528781,35718900,-0.478631,DOWN
2016-12-14,56.691578,57.388013,56.555907,56.981005,30352700,-0.159796,DOWN
...,...,...,...,...,...,...,...
2025-02-28,396.989990,397.630005,386.570007,392.660004,32845700,-2.187959,DOWN
2025-03-03,388.489990,398.820007,386.160004,398.820007,23007700,0.030878,UP
2025-03-04,388.609985,392.579987,381.000000,383.399994,29342900,3.094610,UP


# Simple Investment Strategy

### Create Strategy class

In [3]:
from backtesting import Strategy, Backtest

/home/codespace/.python/current/lib/python3.12/site-packages/backtesting/_plotting.py:54: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [4]:
class Regression(Strategy):
    def init(self):
        self.model = model_dt
        self.already_bought = False

    def next(self):
        explanatory_today = self.data.df.iloc[[-1], :]
        forecast_tomorrow = self.model.predict(explanatory_today)[0]
        
        if forecast_tomorrow > 1 and self.already_bought == False:
            self.buy()
            self.already_bought = True
        elif forecast_tomorrow < -5 and self.already_bought == True:
            self.sell()
            self.already_bought = False
        else:
            pass

### Create Backtest class

In [5]:
df_explanatory = df[['Open', 'High', 'Low', 'Close', 'Volume']].copy()

In [6]:
bt = Backtest(df_explanatory, Regression,
              cash=10000, commission=.002, exclusive_orders=True)

### Run backtesting with specific values

In [7]:
model_dt.predict(df_explanatory)

array([0.10265827, 0.10265827, 0.10265827, ..., 0.23656795, 0.23656795,
       0.23656795], shape=(2071,))

In [8]:
results = bt.run()

### Interpret backtesting results

In [9]:
results.to_frame(name='Values').loc[:'Return [%]']

,Values
Start,2016-12-08 00:00:00
End,2025-03-06 00:00:00
Duration,3010 days 00:00:00
Exposure Time [%],92.708836
Equity Final [$],76374.462719
Equity Peak [$],89289.875801
Commissions [$],2679.450369
Return [%],663.744627


## Parametrize the Investment Strategy

### Create Strategy class

In [10]:
class Regression(Strategy):
    
    limit_buy = 1
    limit_sell = -5
    
    def init(self):
        self.model = model_dt
        self.already_bought = False

    def next(self):
        explanatory_today = self.data.df.iloc[[-1], :]
        forecast_tomorrow = self.model.predict(explanatory_today)[0]
        
        if forecast_tomorrow > self.limit_buy and self.already_bought == False:
            self.buy()
            self.already_bought = True
        elif forecast_tomorrow < self.limit_sell and self.already_bought == True:
            self.sell()
            self.already_bought = False
        else:
            pass

### Create Backtest class

In [11]:
bt = Backtest(df_explanatory, Regression,
              cash=10000, commission=.002, exclusive_orders=True)

### Optimize backtesting with multiple combinations

In [12]:
list_limits_buy = list(range(0, 11, 1))

In [13]:
list_limits_buy

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [14]:
list_limits_sell = list(range(0, -11, -1))

In [15]:
list_limits_sell

[0, -1, -2, -3, -4, -5, -6, -7, -8, -9, -10]

In [16]:
%%time

results = bt.optimize(
    limit_buy = list_limits_buy, limit_sell = list_limits_sell,
    maximize='Return [%]', return_heatmap=True
)

CPU times: user 2.26 s, sys: 27.7 ms, total: 2.29 s
Wall time: 2min 58s


### [ ] Interpret backtesting results

In [17]:
results_heatmap = results[1]

In [18]:
df_results_heatmap = results_heatmap.reset_index()

In [19]:
dff = df_results_heatmap.pivot(
    index='limit_buy', columns='limit_sell', values='Return [%]')

In [20]:
dff

limit_sell,-10,-9,-8,-7,-6,-5,-4,-3,-2,-1,0
limit_buy,,,,,,,,,,,
0,573.064137,573.064137,596.820989,612.888800,815.824238,896.966145,1433.738916,2636.745134,4157.111686,12573.607212,12139.219705
1,486.242401,486.242401,553.990316,569.008729,676.840920,663.744627,1074.206972,633.462047,692.831459,726.985172,33.184945
2,367.757922,367.757922,422.561091,434.910123,508.248479,495.702641,785.033119,289.681592,418.570941,193.493471,-22.888468
3,367.757922,367.757922,240.944340,197.715727,92.124387,103.170107,-77.948775,-74.832405,-70.366446,-72.870509,-84.364676
4,276.672970,276.672970,106.167200,79.886835,28.941312,68.516308,-84.976659,-82.299316,-82.807223,-85.638069,-85.513815
5,276.672970,276.672970,106.167200,79.886835,40.335488,85.839249,-86.121907,-82.769659,-83.295937,-86.603169,-87.659318
6,137.854904,137.854904,29.544350,16.541834,-8.418328,21.113668,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000
7,177.273935,177.273935,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000
8,161.921912,161.921912,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000


In [21]:
dff.sort_index(axis=1, ascending=False)


limit_sell,0,-1,-2,-3,-4,-5,-6,-7,-8,-9,-10
limit_buy,,,,,,,,,,,
0,12139.219705,12573.607212,4157.111686,2636.745134,1433.738916,896.966145,815.824238,612.888800,596.820989,573.064137,573.064137
1,33.184945,726.985172,692.831459,633.462047,1074.206972,663.744627,676.840920,569.008729,553.990316,486.242401,486.242401
2,-22.888468,193.493471,418.570941,289.681592,785.033119,495.702641,508.248479,434.910123,422.561091,367.757922,367.757922
3,-84.364676,-72.870509,-70.366446,-74.832405,-77.948775,103.170107,92.124387,197.715727,240.944340,367.757922,367.757922
4,-85.513815,-85.638069,-82.807223,-82.299316,-84.976659,68.516308,28.941312,79.886835,106.167200,276.672970,276.672970
5,-87.659318,-86.603169,-83.295937,-82.769659,-86.121907,85.839249,40.335488,79.886835,106.167200,276.672970,276.672970
6,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,21.113668,-8.418328,16.541834,29.544350,137.854904,137.854904
7,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,177.273935,177.273935
8,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,-100.000000,161.921912,161.921912


## DataFrame heatmaps for better reporting

In [22]:
dff.sort_index(axis=1, ascending=False)\
    .style.format(precision=0)\
    .background_gradient()

limit_sell,0,-1,-2,-3,-4,-5,-6,-7,-8,-9,-10
limit_buy,,,,,,,,,,,
0,12139,12574,4157,2637,1434,897,816,613,597,573,573
1,33,727,693,633,1074,664,677,569,554,486,486
2,-23,193,419,290,785,496,508,435,423,368,368
3,-84,-73,-70,-75,-78,103,92,198,241,368,368
4,-86,-86,-83,-82,-85,69,29,80,106,277,277
5,-88,-87,-83,-83,-86,86,40,80,106,277,277
6,-100,-100,-100,-100,-100,21,-8,17,30,138,138
7,-100,-100,-100,-100,-100,-100,-100,-100,-100,177,177
8,-100,-100,-100,-100,-100,-100,-100,-100,-100,162,162


import numpy as np

In [24]:
import numpy as np

dff.sort_index(axis=1, ascending=False)\
    .style.format(precision=0)\
    .background_gradient(vmin=np.nanmin(dff), vmax=np.nanmax(dff))

limit_sell,0,-1,-2,-3,-4,-5,-6,-7,-8,-9,-10
limit_buy,,,,,,,,,,,
0,12139,12574,4157,2637,1434,897,816,613,597,573,573
1,33,727,693,633,1074,664,677,569,554,486,486
2,-23,193,419,290,785,496,508,435,423,368,368
3,-84,-73,-70,-75,-78,103,92,198,241,368,368
4,-86,-86,-83,-82,-85,69,29,80,106,277,277
5,-88,-87,-83,-83,-86,86,40,80,106,277,277
6,-100,-100,-100,-100,-100,21,-8,17,30,138,138
7,-100,-100,-100,-100,-100,-100,-100,-100,-100,177,177
8,-100,-100,-100,-100,-100,-100,-100,-100,-100,162,162
